# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [85]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict

from langchain_core.output_parsers import StrOutputParser

In [7]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.6)

# Justification: we want the model to be creative in writing descriptions, so the temperature should be higher - but we still want the description to make sense

In [8]:
prompt = ChatPromptTemplate.from_template( "Write a product description for {product} in 50 words or less"
 
)

In [57]:
# Old code
#chain = LLMChain(llm=llm, prompt=prompt)

# New code:
chain = prompt | llm

In [58]:
product = "Waterproof Phone Pouch"
# Old code
#chain.run(product)

# New code:
chain.invoke(product)

AIMessage(content='One possible solution to creating a waterproof phone pouch is to use a combination of waterproof materials such as PVC or TPU, along with a secure sealing mechanism such as a ziplock or velcro closure. \n\nHere are the imperative steps to create a waterproof phone pouch:\n\n1. Choose a waterproof material such as PVC or TPU that is durable and flexible.\n2. Cut the material to the desired size and shape to fit your phone.\n3. Fold the material in half, with the waterproof side facing outwards.\n4. Use a heat sealer or waterproof adhesive to seal the sides and bottom of the pouch, leaving the top open.\n5. Add a secure closure mechanism such as a ziplock or velcro to ensure a tight seal.\n6. Test the pouch by placing a paper towel inside and submerging it in water to check for any leaks.\n7. Once the pouch is confirmed to be waterproof, insert your phone and seal it securely before using it near water.\n\nBy following these steps and choosing the right materials, you 

## SimpleSequentialChain

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write a product description for {product} in 50 words or less"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [22]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Summarize {first_prmpt} in the style of shakespeare"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [23]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [24]:
#overall_simple_chain.run(product)

# New code:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Keep your phone safe and dry with our Waterproof Phone Pouch. Perfect for any outdoor activity, this pouch is 100% waterproof and allows you to use your touchscreen while keeping your phone protected. With an adjustable neck strap, you can take your phone wherever you go without worry.
Fear not the elements, for with our Waterproof Phone Pouch,
Thy precious device shall be safe and dry, untouched by such touch.
A marvel of technology, 100% waterproof it stands,
Allowing thee to use thy touchscreen with nimble hands.

With adjustable neck strap, thou can carry it with ease,
Into the wild and untamed lands, wherever thou please.
So fear not, dear traveller, take thy phone with thee,
Protected and secure, as it should be.

> Finished chain.


'Fear not the elements, for with our Waterproof Phone Pouch,\nThy precious device shall be safe and dry, untouched by such touch.\nA marvel of technology, 100% waterproof it stands,\nAllowing thee to use thy touchscreen with nimble hands.\n\nWith adjustable neck strap, thou can carry it with ease,\nInto the wild and untamed lands, wherever thou please.\nSo fear not, dear traveller, take thy phone with thee,\nProtected and secure, as it should be.'

**Repeat the above twice for different products**

## SequentialChain

In [25]:
from langchain.chains import SequentialChain

In [37]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Tell me which product the following review describes: {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key= "Review" #Give a name to your output
                    )


In [36]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the {review} into 10 words or less"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key= "Summary" #give a name to this output
                    )


In [38]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the {Summary} into German"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key= "Translation"
                      )


In [59]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Given the {Summary} and {Translation}, tell me which language the review was translated to"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key= "Language"
                     )


In [60]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables= ["review"],
    output_variables=["Summary", "Translation", "Language"],
    verbose=True
)

In [61]:
review = df.Review[5]
#overall_chain(review)

# New code:
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'Summary': 'Goût médiocre, mousse ne tient pas, possible contrefaçon.',
 'Translation': 'Der Geschmack ist mittelmäßig, der Schaum hält nicht, möglicherweise eine Fälschung.',
 'Language': 'The review was translated to French and German.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [112]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

default_template = """You are a helpful assistant. 
Question: {input}
Provide a clear, helpful answer."""

In [118]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "history", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },

    {
        "name": "DEFAULT",
        "description": "Good for general questions",
        "prompt_template": default_template
    }
]


In [114]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [115]:
llm = ChatOpenAI(temperature=0)

In [119]:
expert_chains = {}
for info in prompt_infos:
    prompt = ChatPromptTemplate.from_template(template=info["prompt_template"])
    chain = prompt | llm | StrOutputParser()
    expert_chains[info["name"]] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [120]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [121]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_prompt = ChatPromptTemplate.from_template(MULTI_PROMPT_ROUTER_TEMPLATE)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

class RouterOutput(TypedDict):
    destination: Literal["pysics", "math", "history", "computer_science", "DEFAULT"]

router_chain = router_prompt | llm.with_structured_output(RouterOutput) | itemgetter("destination")

def route_to_chain (inputs: dict) -> str:
    destination = inputs["destination"]
    query = inputs["query"]
    return expert_chains[destination].invoke({"input": query})

chain = (
    {
        "query": RunnablePassthrough(),
        "destination": lambda x:
router_chain.invoke({"destinations": destinations_str,
"question": x})
    }
    | RunnableLambda(route_to_chain)
)


In [122]:
if __name__ == "__main__":
    # Test with different types of questions
    questions = [
        "What is black body radiation?", 
        "What is 2+2",      
        "What caused World War I?",       
        "Explain Big O notation",     
        "What is the forecast for today?", 
        "Explain what principle component analysis is using linear algebra",
       # "What are cells?" #still breaks it 
    ]
    
    for question in questions:
        print(f"\nQuestion: {question}")
        result = chain.invoke(question)
        print(f"Answer: {result}\n")
        print("-" * 50)


Question: What is black body radiation?
Answer: Black body radiation refers to the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across a wide range of wavelengths. The spectrum of black body radiation is continuous and follows a specific distribution known as Planck's law. This phenomenon is important in understanding the behavior of objects at high temperatures, such as stars, and is also used in various fields of science and engineering.

--------------------------------------------------

Question: What is 2+2
Answer: 2 + 2 equals 4.

--------------------------------------------------

Question: What caused World War I?
Answer: World War I was caused by a combination of factors, including militarism, alliances, imperialism, and nationalism. The assassination of Archduke Franz Ferdinand of Austria-Hungary in 1914 was the immediate trigger that set off a

In [123]:
chain.invoke("What is black body radiation?")

"Black body radiation refers to the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across a wide range of wavelengths. The spectrum of black body radiation is continuous and follows a specific distribution known as Planck's law. This phenomenon is important in understanding the behavior of objects at high temperatures, such as stars, and is also used in various fields of science and engineering."

In [125]:
chain.invoke("what is 2 + 2")

'2 + 2 equals 4.'

In [126]:
chain.invoke("Why does every cell in our body contain DNA?")

'Every cell in our body contains DNA because DNA carries the genetic information that determines our unique characteristics and functions. This genetic information is essential for the growth, development, and functioning of our cells, tissues, and organs. DNA also plays a crucial role in passing on genetic traits from one generation to the next. Having DNA in every cell ensures that the genetic instructions are available for proper cell division, protein synthesis, and overall maintenance of our body.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**